Data Wrangling Notebook for SY Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [1]:
import pandas as pd
from pandas import Series
import numpy as np
import re
import uuid 

Import SY Data

In [2]:
sy_data = pd.read_csv("./../Original_Data/futres_sy_original.csv")

Rename Columns

In [4]:
sy_data = sy_data.rename(columns={'ID': 'individualID',
                                  'Taxa':'scientificName',
                                  'Measurements (NAME;NAME;NAME...) divided by ;':'measurementType',
                                  'Values (mm;mm;mm...) divided by ;':'measurementValue',
                                  'Gender': 'sex',
                                  'Age appearance':'lifeStage'})

Clean up lifeStage column

In [5]:
# Create Filters
adult = sy_data['lifeStage']=="ADL"
juvenile = sy_data['lifeStage']=="JUV"

# Assign correct terms using filters
sy_data['lifeStage'][adult] = "adult"
sy_data['lifeStage'][juvenile] = "juvenile"

<ipython-input-5-b12a5b877cf9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data['lifeStage'][adult] = "adult"
<ipython-input-5-b12a5b877cf9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data['lifeStage'][juvenile] = "juvenile"


Clean up sex column

In [6]:
male = sy_data['sex']=="M"
sy_data["sex"][male==True]="male"
sy_data['sex'][male==False]=""

<ipython-input-6-52260372d541>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data["sex"][male==True]="male"
<ipython-input-6-52260372d541>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sy_data['sex'][male==False]=""


In [8]:
sy_data

,individualID,Identifiable (si/astilla/huesoplano/fragmin),Site,DATUM,UTM E,UTM N,Unit,Level,Layer,Gral category,...,13-alteration degree,13-alteration depth,observations,pathology/trauma,photo,comments,MEASUREMENTS,PROTOCOL,measurementType,measurementValue
0,2,si,H13,WGS84 19S,271261,6417165,P12,4,3,Tetrapodo,...,NaN,NaN,lobo marino,NaN,s/f,mismo metacarpo,SI,L'Hereux y Borella 2011,Bd;Dd,"29,5;17,2"
1,8,si,H36/37,WGS84 19S,271562,6417478,P3,1,2,Tetrapodo,...,NaN,NaN,falange nutria,NaN,PICT0004,"mustelido posible lontra felina, alta similitu...",SI,Von den Driesch (1976),GL;Bp;Bd;Dp;Dd,"14;4,5;4;4;2"
2,12,si,H12,WGS84 19S,272041,6417644,P5,3,2A,Tetrapodo,...,NaN,NaN,roedor grande,NaN,s/f,NaN,SI,Von den Driesch (1976),PL;BFcr;HFcr,"7;4,5;2,2"
3,17,si,H36/37,WGS84 19S,271562,6417478,PNE,4,2,Tetrapodo,...,NaN,NaN,mustelido,NaN,s/f,NaN,SI,Von den Driesch (1976),PL,"13,1"
4,20,si,H13,WGS84 19S,271261,6417165,P12,5,3A,Tetrapodo,...,1.0,superficial,falange lobo marino,NaN,s/f,NaN,SI,L'Hereux y Borella 2011,"Bp,Dp","27,5;22,2"
5,24,si,H13,WGS84 19S,271261,6417165,P12,3,3,Tetrapodo,...,NaN,NaN,NaN,NaN,s/f,NaN,SI,L'Hereux y Borella 2011,Dd,19
6,45,si,H13,WGS84 19S,271261,6417165,P12,5,3A,Tetrapodo,...,NaN,NaN,NaN,NaN,esp4 - esp5,especimen semiarticulado de posible sturnella ...,SI,Von den Driesch (1976),GL;La;Dip;SC;Bd;Dd,"47,2;46,8;7;2,9;4;3,85"
7,46,si,H13,WGS84 19S,271261,6417165,P12,5,3A,Tetrapodo,...,NaN,NaN,"fractura postdept, medidas en * estan en el es...",NaN,esp4 - esp5,especimen semiarticulado de posible sturnella ...,SI,Von den Driesch (1976),GL;La;Dip;SC;Bd*;Dd*,"47;46,3;7,1;2,5"
8,47,si,H13,WGS84 19S,271261,6417165,P12,5,3A,Tetrapodo,...,NaN,NaN,"mismo especimen que 46, medidas incluidas en l...",NaN,esp4 - esp5,especimen semiarticulado de posible sturnella ...,SI,Von den Driesch (1976),Bd;Dd,"4;3,9"
9,48,si,H13,WGS84 19S,271261,6417165,P12,5,3A,Tetrapodo,...,NaN,NaN,NaN,NaN,esp4 - esp5,especimen semiarticulado de posible sturnella ...,SI,Von den Driesch (1976),GL;Lm;Bp;Dp;SC;Bd;Dd,"26,5;25;4,9;2,9;2;5,25;3"


Check is length of measurementType and measurementValue are the same 

In [10]:



def matching_size(row):
    """function to match size of measurement type and value"""
    mtype = row["measurementType"].split(';')
    mval = row["measurementValue"].split(';')
    if len(mtype) == len(mval):
        linesplit(row)
    else:
        print ("Error: Size Mismatch")
        print(row.name)
        
def line_split(row)
        
sy_data = sy_data.apply(matching_size,  axis =1)


Size Match
Size Match
Size Match
Size Match
Error: Size Mismatch
4
Size Match
Size Match
Error: Size Mismatch
7
Size Match
Size Match
Size Match
Size Match
Size Match
Size Match
Size Match
Size Match
Size Match
Size Match
Size Match
Size Match


Expand data using ; as delimiter